   # Education Project 


   <img src='data/education_image.jpg' width="900">
   
   **Credit:**  [wsimag](https://wsimag.com/culture/60264-education-in-venezuela-the-americas-and-the-world)



In [1]:
# Load relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import warnings

sns.set(style='ticks')

warnings.filterwarnings("ignore")  # Suppress all warnings

# Introduction

## Business Context
Research shows that high-poverty areas disproportionally educate children of color. The chances of ending up in a high-poverty or high-minority school are highly determined by a student’s race/ethnicity and social class. For instance, African American and Hispanic students—even if they are not poor—are much more likely than white or Asian students to be in high-poverty schools.

There is a growing body of evidence that shows increased investment on education returns better outcomes and that the positive effects are even greater among low-income students. On the other hand, it costs more to educate low-income students and provide them with a robust education capable of overcoming their initial disadvantages.


### Goals
1. Understand the current demographics of wealthy to high-poverty schools across the state of California.
2. Identify how much funding is available per pupil in wealthy vs high-poverty areas.
3. Discover what the return on investment per student based on race and socioeconomic status is.
4. Determine the ideal capital to provide a robust education to students of color and/or in high-poverty areas.

#### Predictive modeling
Identify the optimal capital cost per child to maximize outcome according to ethnicity.


# Data wrangling

The process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics.

## Extracting and cleaning relevant data

Let's start looking at the datasets!

### Assessment Data

- It contains assessment data for the Smarter Balance Summative Assessment (2018-2019) for the state of California.

- Legend types can be found here: https://caaspp-elpac.cde.ca.gov/caaspp/research_fixfileformat19

In [2]:
# loa datafile
df_all = pd.read_csv('large_data/sb_ca2019_all_csv_v4.txt')

In [3]:
# create dataset containing district level data
df_district = df_all[df_all['District Code'] == 00000]

In [4]:
# create dataset containing school level data
df_school = df_all.drop(df_all[df_all['School Code'] == 0].index)
#df_school

In [5]:
# check columns' names
df_school.columns

Index(['County Code', 'District Code', 'School Code', 'Filler', 'Test Year',
       'Subgroup ID', 'Test Type', 'Total Tested At Entity Level',
       'Total Tested with Scores', 'Grade', 'Test Id',
       'CAASPP Reported Enrollment', 'Students Tested', 'Mean Scale Score',
       'Percentage Standard Exceeded', 'Percentage Standard Met',
       'Percentage Standard Met and Above', 'Percentage Standard Nearly Met',
       'Percentage Standard Not Met', 'Students with Scores',
       'Area 1 Percentage Above Standard', 'Area 1 Percentage Near Standard',
       'Area 1 Percentage Below Standard', 'Area 2 Percentage Above Standard',
       'Area 2 Percentage Near Standard', 'Area 2 Percentage Below Standard',
       'Area 3 Percentage Above Standard', 'Area 3 Percentage Near Standard',
       'Area 3 Percentage Below Standard', 'Area 4 Percentage Above Standard',
       'Area 4 Percentage Near Standard', 'Area 4 Percentage Below Standard'],
      dtype='object')

In [6]:
# check data type
df_school.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3013079 entries, 1888 to 3576490
Data columns (total 32 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   County Code                        int64  
 1   District Code                      int64  
 2   School Code                        int64  
 3   Filler                             float64
 4   Test Year                          int64  
 5   Subgroup ID                        int64  
 6   Test Type                          object 
 7   Total Tested At Entity Level       object 
 8   Total Tested with Scores           object 
 9   Grade                              int64  
 10  Test Id                            int64  
 11  CAASPP Reported Enrollment         object 
 12  Students Tested                    object 
 13  Mean Scale Score                   object 
 14  Percentage Standard Exceeded       object 
 15  Percentage Standard Met            object 
 16  Percentage Stan

#### Before merging:
- Filter variables of interest;
- Rearrange the data to have one variable per column;

The assessment dataset contains a lot of demographic information in the subgroup ID column. Let's reorganize to have one variable per column and keep only the variables of interest!

I'm working with the Smater Balanced Assessments for English Language Arts/Literacy and Mathematics (SB). Test ID 1 and 2. More info about the test can be found here: https://www.caaspp.org/administration/about/testing/index.html

#### Subgroup ID 



In [7]:
# filter demographic of interest from Subgroup ID
subgroup_id = [3, 4, 50, 51, 52, 53, 90, 91, 92, 93, 94, 220, 221, 222, 223, 
               224, 225, 226, 227, 200, 201, 202, 203, 204, 205, 206, 207]

df_school_id = df_school[df_school['Subgroup ID'].isin(subgroup_id)]

In [8]:
# drop columns with redundant information or not useful
df_school_id = df_school_id.drop(columns = ['Filler', 'Test Year', 'Test Type', 'Grade', 'County Code',
                             'District Code', 'Total Tested At Entity Level', 'Total Tested with Scores',
                             'Area 1 Percentage Above Standard', 'Area 1 Percentage Near Standard',
                             'Area 1 Percentage Below Standard', 'Area 2 Percentage Above Standard',
                             'Area 2 Percentage Near Standard', 'Area 2 Percentage Below Standard',
                             'Area 3 Percentage Above Standard', 'Area 3 Percentage Near Standard',
                             'Area 3 Percentage Below Standard', 'Area 4 Percentage Above Standard',
                             'Area 4 Percentage Near Standard', 'Area 4 Percentage Below Standard'])
df_school_id

,School Code,Subgroup ID,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
1889,112607,3,1,43,42,2538.4,9.52,21.43,30.95,40.48,28.57,42
1890,112607,4,1,47,43,2544.0,4.76,33.33,38.10,33.33,28.57,42
1895,112607,51,1,90,85,2541.2,7.14,27.38,34.52,36.90,28.57,84
1896,112607,53,1,90,85,2541.2,7.14,27.38,34.52,36.90,28.57,84
1903,112607,90,1,23,23,2568.3,8.70,39.13,47.83,26.09,26.09,23
...,...,...,...,...,...,...,...,...,...,...,...,...
3576486,5838305,222,2,*,*,NaN,*,*,*,*,*,*
3576487,5838305,223,2,*,*,NaN,*,*,*,*,*,*
3576488,5838305,224,2,19,19,NaN,0.00,15.79,15.79,42.11,42.11,19
3576489,5838305,226,2,57,54,NaN,7.41,24.07,31.48,29.63,38.89,54


In [9]:
df_school_id.isnull().sum()

School Code                               0
Subgroup ID                               0
Test Id                                   0
CAASPP Reported Enrollment                0
Students Tested                           0
Mean Scale Score                     402787
Percentage Standard Exceeded              0
Percentage Standard Met                   0
Percentage Standard Met and Above         0
Percentage Standard Nearly Met            0
Percentage Standard Not Met               0
Students with Scores                      0
dtype: int64

## Next: 
1. Transform demographic information contained into subgroup id (rows) to one variable per column.
2. Do the same with other demographic information contained in subgroup id column.

In [10]:
legend = pd.read_csv('data/Subgroups.txt')
legend

,Demographic ID,Demographic ID Num,Demographic Name,Student Group
0,1,1,All Students,All Students
1,3,3,Male,Gender
2,4,4,Female,Gender
3,6,6,Fluent English proficient and English only,English-Language Fluency
4,7,7,Initial fluent English proficient (IFEP),English-Language Fluency
5,8,8,Reclassified fluent English proficient (RFEP),English-Language Fluency
6,28,28,Migrant education,Migrant
7,31,31,Economically disadvantaged,Economic Status
8,50,50,Military,Military Status
9,51,51,Not Military,Military Status


In [11]:
# create new column with label for economically disadvataged/not students 
def define_label(i):
    if i in (200, 201, 202, 203, 204, 205, 206, 207):
        return 'Disadvantaged'
    if i in (220, 221, 222, 223, 224, 225, 226, 227):
        return 'Not Disadvantaged'

df_school_id['Disadvantaged'] = df_school_id['Subgroup ID'].apply(define_label)

In [12]:
# repace code with labels
# male and female 
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=3, value='Male')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=4, value='Female')

# Military and not military
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=50, value='Military')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=51, value='Not Military')

# Homeless and not homeless
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=52, value='Homeless')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=53, value='Not Homeless')

# Parents education
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=90, value='Not High School Grad')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=91, value='High School Grad')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=92, value='Some college')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=93, value='College Grad')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=94, value='Graduate School')


In [13]:
# replace code with actual ethnicity
# Not Disadvantaged
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=200, value='Black')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=201, value='American Indian')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=202, value='Asian')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=203, value='Filipino')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=204, value='Hispanic')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=205, value='Pacific Islander')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=206, value='White')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=207, value='Two or More Races')

# Disadvantaged
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=220, value='Black')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=221, value='American Indian')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=222, value='Asian')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=223, value='Filipino')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=224, value='Hispanic')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=225, value='Pacific Islander')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=226, value='White')
df_school_id['Subgroup ID'] = df_school_id['Subgroup ID'].replace(to_replace=227, value='Two or More Races')

In [14]:
df_school_id

,School Code,Subgroup ID,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Disadvantaged
1889,112607,Male,1,43,42,2538.4,9.52,21.43,30.95,40.48,28.57,42,None
1890,112607,Female,1,47,43,2544.0,4.76,33.33,38.10,33.33,28.57,42,None
1895,112607,Not Military,1,90,85,2541.2,7.14,27.38,34.52,36.90,28.57,84,None
1896,112607,Not Homeless,1,90,85,2541.2,7.14,27.38,34.52,36.90,28.57,84,None
1903,112607,Not High School Grad,1,23,23,2568.3,8.70,39.13,47.83,26.09,26.09,23,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3576486,5838305,Asian,2,*,*,NaN,*,*,*,*,*,*,Not Disadvantaged
3576487,5838305,Filipino,2,*,*,NaN,*,*,*,*,*,*,Not Disadvantaged
3576488,5838305,Hispanic,2,19,19,NaN,0.00,15.79,15.79,42.11,42.11,19,Not Disadvantaged
3576489,5838305,White,2,57,54,NaN,7.41,24.07,31.48,29.63,38.89,54,Not Disadvantaged


---------

### Entities Data

- It contains information such as school and district name, as well as zip code and relevant codes that will allow merging with the assessment data. 
- It comes from the California Assessment of Student Performance and Progress.

Dataset number of rows match current information about the state of CA:

- There are ~ 1,040 school districts in California. 
    - The entities_dist dataset contains 1,087 rows.
- There are ~ 10,588 schools in California. 
    - The df_entities dataset contains 10,300 rows.

In [15]:
df_entities = pd.read_csv('data/sb_ca2019entities_csv.txt')

In [16]:
# create dataset containing entities data at district level
entities_dist = df_entities[df_entities['School Code'] == 0]


In [17]:
# create dataset containing entities data at school level 
df_entities = df_entities.drop(df_entities[df_entities['School Code'] == 0].index) # drop district level data

In [18]:
df_entities['County Name'].unique()

array(['San Diego', 'San Francisco', 'San Joaquin', 'San Luis Obispo',
       'Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'San Mateo',
       'Santa Barbara', 'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra',
       'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Tehama',
       'Trinity', 'Tulare', 'Tuolumne', 'Glenn', 'Humboldt', 'Imperial',
       'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles',
       'Ventura', 'Yolo', 'Yuba', 'Madera', 'Marin', 'Mariposa',
       'Mendocino', 'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa',
       'Nevada', 'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento',
       'San Benito', 'San Bernardino'], dtype=object)

In [19]:
# drop columns with redundant information or not of use 
df_entities = df_entities.drop(columns = ['Filler', 'Type Id', 'County Code', 'District Code', 'District Name', 'County Name'])
df_entities

,School Code,Test Year,School Name,Zip Code
0,114686,2019,Ocean Air,92130
1,6038111,2019,Del Mar Heights Elementary,92014
2,6088983,2019,Del Mar Hills Elementary,92014
3,6110696,2019,Carmel Del Mar Elementary,92130
4,6115620,2019,Ashley Falls Elementary,92130
...,...,...,...,...
11383,136747,2019,California Academy of Sports Science,91764
11384,138313,2019,University Prep,91764
11385,6038095,2019,Dehesa Elementary,92019
11386,6119564,2019,Dehesa Charter,92026


--------

## Merge df_school_id with df_entities

This merge adds school name, zipcode, and test year to the main df.

In [20]:
# merge dfs on school code
df_merge = pd.merge(df_entities, df_school_id, on='School Code')

df_merge

,School Code,Test Year,School Name,Zip Code,Subgroup ID,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Disadvantaged
0,114686,2019,Ocean Air,92130,Male,1,47,45,2535.5,77.78,17.78,95.56,4.44,0.00,45,None
1,114686,2019,Ocean Air,92130,Female,1,46,46,2522.0,73.91,17.39,91.30,4.35,4.35,46,None
2,114686,2019,Ocean Air,92130,Military,1,*,*,*,*,*,*,*,*,*,None
3,114686,2019,Ocean Air,92130,Not Military,1,92,90,2528.5,75.56,17.78,93.33,4.44,2.22,90,None
4,114686,2019,Ocean Air,92130,Not Homeless,1,93,91,2528.7,75.82,17.58,93.41,4.40,2.20,91,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494617,110114,2019,Sycamore Ridge,92130,Asian,2,121,121,NaN,85.12,7.44,92.56,4.96,2.48,121,Not Disadvantaged
1494618,110114,2019,Sycamore Ridge,92130,Filipino,2,*,*,NaN,*,*,*,*,*,*,Not Disadvantaged
1494619,110114,2019,Sycamore Ridge,92130,Hispanic,2,40,40,NaN,37.50,15.00,52.50,30.00,17.50,40,Not Disadvantaged
1494620,110114,2019,Sycamore Ridge,92130,White,2,129,127,NaN,57.48,28.35,85.83,8.66,5.51,127,Not Disadvantaged


In [21]:
df_merge.isnull().sum()

School Code                               0
Test Year                                 0
School Name                               0
Zip Code                                  0
Subgroup ID                               0
Test Id                                   0
CAASPP Reported Enrollment                0
Students Tested                           0
Mean Scale Score                     402787
Percentage Standard Exceeded              0
Percentage Standard Met                   0
Percentage Standard Met and Above         0
Percentage Standard Nearly Met            0
Percentage Standard Not Met               0
Students with Scores                      0
Disadvantaged                        797900
dtype: int64

----------

### Expenses Data

- It contains the current cost of education for school districts in California.
- The dataset contains variables such school district expense average daily attendance cost for the academic year 2018-2019.

In [22]:
df_expenses = pd.read_excel('data/currentexpense1819.xlsx')

In [23]:
df_expenses = df_expenses.drop(df_expenses.index[[0,1,2,3,4,5,6,7,8]])

In [24]:
new_header = df_expenses.iloc[0] #grab the first row for the header
df_expenses = df_expenses[1:] #take the data less the header row
df_expenses.columns = new_header #set the header row as the df header

In [25]:
df_expenses

9,CO Code,District Code,District,EDP 365,Current\nExpense ADA,Current\nExpense Per ADA,LEA Type
10,01,61119,Alameda Unified,117225882.5,8968.85,13070.335941,Unified
11,01,61127,Albany City Unified,46611059.59,3544.52,13150.175366,Unified
12,01,61143,Berkeley Unified,159457818.49,9356.44,17042.573724,Unified
13,01,61150,Castro Valley Unified,102239937.34,8940.2,11435.978763,Unified
14,01,61168,Emery Unified,12504023.21,681.82,18339.185137,Unified
...,...,...,...,...,...,...,...
944,58,72728,Camptonville Elementary,776334.87,44.68,17375.444718,Elementary
945,58,72736,Marysville Joint Unified,107389549.48,9072.18,11837.23752,Unified
946,58,72744,Plumas Lake Elementary,12851169.64,1283.04,10016.187835,Elementary
947,58,72751,Wheatland Elementary,15925495.9,1236.92,12875.121997,Elementary


---------

### Enrollment Dataset, Full-Time Equivalent Teacher, and Pupil/Teacher Ratio
- It contains total enrollment per school for the academic year 2018-2019 in California.
- Data comes from the National Center for Education Statistics.

In [26]:
# load datafile
df_enrollment = pd.read_csv('data/ELSI_total_enrollment_.csv')
df_enrollment

,School Name,State Name [Public School] Latest available year,School ID - NCES Assigned [Public School] Latest available year,Agency ID - NCES Assigned [Public School] Latest available year,Total Students All Grades (Excludes AE) [Public School] 2018-19,Full-Time Equivalent (FTE) Teachers [Public School] 2018-19,Pupil/Teacher Ratio [Public School] 2018-19
0,21ST CENTURY LEARNING INSTITUTE,CALIFORNIA,"=""060429013779""","=""0604290""",88,3.60,24.44
1,A PLACE TO GROW,California,"=""062827013394""","=""0628270""",†,–,–
2,A. E. ARNOLD ELEMENTARY,California,"=""061044001166""","=""0610440""",739,27.00,27.37
3,A. G. COOK ELEMENTARY,California,"=""061488001834""","=""0614880""",366,16.00,22.88
4,A. G. CURRIE MIDDLE,California,"=""064015006636""","=""0640150""",611,25.30,24.15
...,...,...,...,...,...,...,...
10436,ZUPANIC HIGH,California,"=""063237010019""","=""0632370""",70,3.00,23.33
10437,Data Source: U.S. Department of Education Nati...,NaN,NaN,NaN,NaN,NaN,NaN
10438,† indicates that the data are not applicable.,NaN,NaN,NaN,NaN,NaN,NaN
10439,– indicates that the data are missing.,NaN,NaN,NaN,NaN,NaN,NaN


---------

### Total Revenue

- It contains total revenue per school district in California for the academic year 2018-2019.
- Revenue comes from local, state and federal sources.

In [27]:
df_revenue = pd.read_csv('data/ELSI_csv_export_revenue.csv')

In [28]:
df_revenue

,Agency Name,State Name [District] Latest available year,Fall Membership (V33) [District Finance] 2016-17,Total Revenue - Local Sources (TLOCREV) [District Finance] 2016-17,Total General Revenue (TOTALREV) [District Finance] 2016-17,Total Revenue - State Sources (TSTREV) [District Finance] 2016-17,Total Revenue - Federal Sources (TFEDREV) [District Finance] 2016-17,Total Current Expenditures - El-Sec Education (TCURELSC) [District Finance] 2016-17,Total Expenditures (TOTALEXP) [District Finance] 2016-17,Total Revenue (TOTALREV) per Pupil (V33) [District Finance] 2016-17,Total Revenue - Local Sources (TLOCREV) per Pupil (V33) [District Finance] 2016-17,Total Revenue - State Sources (TSTREV) per Pupil (V33) [District Finance] 2016-17,Total Revenue - Federal Sources (TFEDREV) per Pupil (V33) [District Finance] 2016-17,Agency ID - NCES Assigned [District] Latest available year
0,ABC UNIFIED,California,20768,52379000,258745000,190473000,15893000,227441000,252804000,12459,2522,9171,765,"=""0601620"""
1,ACALANES UNION HIGH,California,5530,75958000,91214000,13651000,1605000,75057000,84811000,16494,13736,2469,290,"=""0601650"""
2,ACKERMAN CHARTER,California,†,1938000,5686000,3525000,223000,4945000,6018000,†,†,†,†,"=""0601680"""
3,ACTON-AGUA DULCE UNIFIED,California,10016,11199000,35216000,22767000,1250000,28079000,34504000,3516,1118,2273,125,"=""0600001"""
4,ADELANTO ELEMENTARY,California,10288,11183000,127966000,105877000,10906000,111119000,115722000,12438,1087,10291,1060,"=""0601710"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160,YUCAIPA-CALIMESA JOINT UNIFIED,California,9969,23778000,107263000,76380000,7105000,95746000,99066000,10760,2385,7662,713,"=""0643560"""
1161,Data Source: U.S. Department of Education Nati...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1162,† indicates that the data are not applicable.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1163,– indicates that the data are missing.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
